# Optimising simulations for a generic loss function

This notebook showcases optimising a design for some generic loss function $\mathcal{L}$ using an event loop calling **Elmer** or **HFSS**.
As such, this notebook should be run on a machine with the simulation software, KLayout and necessary Python packages installed.
This is similar to Ansys Optimetrics but more general due to supporting arbitrary KQCircuits geometry,
Elmer and any search algorithms and schedulers.
The current implementation requires installing [`ray`](https://docs.ray.io/en/releases-2.0.0/index.html)
```bash
pip install "ray[tune]"==2.1.0
```
and the optimisers generally demand additional manual installation. For example, the example in this notebook employs [`Optuna`](https://optuna.readthedocs.io/en/stable/) and we need to also install:
```bash
pip install optuna==3.0.4 botorch==0.7.3 scikit-learn==1.1.3 plotly==5.11.0
```
Newer versions may work but these are the ones the notebook was written with.

## Workflow

1. Supply your ``Simulation`` class, `sim_parameters`, and `export_parameters` as usual in the [Simulation settings](#simulation-settings)
2. Implement fetching simulation results for your use case and a meaningful loss function $\mathcal{L}$ in the [Loss settings](#loss-settings) section.
3. Specify the variables that can be varied by the optimiser and their ranges in [Optimisation settings](#optimisation-settings). See [Ray Tune - Random Distributions API](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/search_space.html?highlight=quniform#random-distributions-api) for possible distributions.
4. Select your optimiser and its settings. You can also try using a simpler framework, such as, `scipy`, `pytorch`, `tensorflow` etc. See [Ray Tune - Search Algorithms](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html) for algorithms with support out-of-the-box.
5. Run the optimiser (`ray` tuning)
6. View the results! You can manipulate the DataFrame, or use [Tensorboard](https://www.tensorflow.org/tensorboard) to view the data with in the same folder as this notebook:
   ```bash
   pip install tensorboard
   tensorboard --logdir=./ray_results
   ```


The _toy example_ implemented in this notebook is optimising the qubit $C_\Sigma$ ``DoublePads`` to be $65\,\text{fF}$.

In [1]:
import os
import re
import json
import shutil
import subprocess
import importlib
from pathlib import Path
from functools import partial

import pandas as pd
import ray
import ray.air
import ray.air.session
from ray import tune

from kqcircuits.pya_resolver import pya
from kqcircuits.defaults import STARTUPINFO
from kqcircuits.util.geometry_json_encoder import GeometryJsonEncoder
from kqcircuits.simulations.export.ansys.ansys_export import export_ansys
from kqcircuits.simulations.export.elmer.elmer_export import export_elmer
from kqcircuits.simulations.export.simulation_export import export_simulation_oas
from kqcircuits.util.export_helper import create_or_empty_tmp_directory, get_active_or_new_layout

if os.environ.get('KQC_TMP_PATH') is None:
    os.environ['KQC_TMP_PATH'] = os.getcwd()

### Please set the location of your interpreter for `.sh` files

Some examples are given below

In [2]:
# Please set the location of your interpreter for `.sh` files, some examples are given below
sh_interpreter = 'sh'
sh_interpreter = shutil.which('bash') if os.name != 'nt' else 'wsl -e bash'  # use WSL on Windows
sh_interpreter = r'C:\Program Files\Git\bin\bash.exe'

tool = ['ansys', 'elmer'][1]
# This needs to be only as high as the number of licenses when using Ansys
n_workers = 5 if tool == 'elmer' else 1  # first-level parellelisation
n_processes = 4 if tool == 'elmer' else 5  # second-level parellelisation

## Implement helper functions

In [3]:
def create_and_export_sim(params, sim_class, sim_parameters, export_parameters, i):
    """Creates and exports a :class:`.Simulation` with given parameters.

    Arguments:
        params: Iteration-specific parameters to use. Combined with ``sim_parameters``.
        sim_class (Simulation): Given simulation. Should be loadable from a module (not anonymous).
        sim_parameters (dict): Dict of nominal simulation parameters.
        export_parameters (dict): Dict of simulation export parameters. May be for Ansys or Elmer
        i (str): Unique identifier for this iteration.

    Returns:
        dir_path: Path to exported simulations folder.
    """

    # Find corresponding class, has to be done this way to avoid pickling
    module = importlib.import_module(sim_class[0])
    sim_class = getattr(module, sim_class[1])

    dir_path = create_or_empty_tmp_directory(sim_parameters['name'] + f"_opt_{i}")
    export_parameters = {'path': dir_path, **export_parameters}

    # Get layout
    layout = get_active_or_new_layout()

    # Nominal simulation
    simulations = [sim_class(layout, **{
        **sim_parameters,
        **params
    })]

    if 'ansys_tool' in export_parameters:
        export_ansys(simulations, **export_parameters)
    else:
        export_elmer(simulations, **export_parameters)

    # Write oas files
    export_simulation_oas(simulations, dir_path)

    return dir_path

Implement small serializer for DBoxes, as it is often needed in `sim_parameters`.

In [4]:
def dbox_serializer(dbox):
    return GeometryJsonEncoder.encode_geometry(dbox)

def dbox_deserializer(o):
    return pya.DBox(
        pya.DPoint(o[0][0], o[0][1]), pya.DPoint(o[1][1], o[1][1])
    )

ray.util.register_serializer(pya.DBox, serializer=dbox_serializer, deserializer=dbox_deserializer)

## Simulation settings

Specify simulation export settings as normal

In [5]:
# pylint: disable=invalid-name,wrong-import-position
from kqcircuits.simulations.double_pads import DoublePadsSim

sim_class = DoublePadsSim

# Nominal simulation parameters
sim_parameters = {
    'name': re.sub(r'(?<!^)(?=[A-Z])', '_', sim_class.__name__).lower(),  # convert to snake_case
    'use_internal_ports': True,
    'use_ports': True,
    'internal_island_ports': True,
    'box': pya.DBox(pya.DPoint(0,0), pya.DPoint(2000, 2000)),
    'wafer_stack_type': 'planar'
}

#########
# Ansys #
#########

export_parameters_ansys_q3d = {
    'ansys_tool': 'q3d',
    'exit_after_run': True,
    'percent_error': 0.3,
    'minimum_converged_passes': 2,
    'maximum_passes': 20,
}

export_parameters_ansys_epr = {
    'ansys_tool': 'eigenmode',
    'exit_after_run': True,
    'max_delta_f': 0.5,

    # do two passes with tight mesh
    'gap_max_element_length': 20,
    'maximum_passes': 2,
    'minimum_passes': 1,
    'minimum_converged_passes': 1,

    # lossy eigenmode simulation settings
    'n_modes': 1,
    'frequency': 0.5,  # minimum allowed eigenfrequency
    'simulation_flags': ['pyepr'],
    'substrate_loss_tangent': 1e-6,

    # run T1 analysis with pyEPR between simulations
    'intermediate_processing_command': 'python "scripts/t1_estimate.py"',
    'participation_sheet_distance': 5e-3,  # in µm
    'dielectric_surfaces': {
        'layerMA': {
            'tan_delta_surf': 0.001,  # loss tangent
            'th': 4.8e-9,  # thickness
            'eps_r': 10,  # relative permittivity
        },
        'layerMS': {
            'tan_delta_surf': 0.001,
            'th': 0.3e-9,  # estimate worst case
            'eps_r': 10,
        },
        'layerSA': {
            'tan_delta_surf': 0.001,
            'th': 2.4e-9,
            'eps_r': 10,
        }
    },
}

#########
# Elmer #
#########

export_parameters_elmer = {
    'tool': 'capacitance',
    'workflow': {
        'python_executable': 'python',  # use 'kqclib' when using singularity
        'run_gmsh_gui': False,
        'elmer_n_processes': n_processes,  # -1 means all the physical cores
        'gmsh_n_threads': n_processes,
    },
    'gmsh_params': {
        'default_mesh_size': 100.,
        'gap_min_mesh_size': 2.,
        'gap_min_dist': 4.,
        'gap_max_dist': 200.,
        'port_min_mesh_size': 1.,
        'port_min_dist': 4.,
        'port_max_dist': 200.,
        'algorithm': 5,
    }
}

We choose to use Elmer for the capacitance simulations

In [6]:
export_parameters = export_parameters_elmer  # use Elmer
# export_parameters = export_parameters_ansys_epr  # use Ansys + pyEPR

## Loss settings

These need to be adjusted to fit your specific use case.
The optimiser is set to minimise. If you want to maximise, you can modify $\mathcal{L} \to -\mathcal{L}$.
Furthermore, the simplest way to attain a specific value is to set the loss function to
$$
\mathcal{L}(x) = \left| x_\text{desired} - x \right|
$$
This is not differentiable at $\mathcal{L}(x_\text{desired})$, but appears to work decently.
A common alternative is to use the mean squared error ($L_2$-norm).

In [7]:
def load_results_Q(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('Qdata*.csv'))[0]
    df = pd.read_csv(result_file)
    return df.iloc[0].to_dict()


def loss_Q(data):
    r"""Loss function :math:`$\mathcal(L)` to minimum loss :math:`\delta`."""
    Q = data['Q_total']
    return 1 / Q, Q


def load_results_capacitance(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('*_project_results.json'))[0]
    with open(result_file) as fp:
        data = json.load(fp)
    return data


def loss_C_Sigma_two_islands(data, C):
    r""" Data argument is a dict with `CMatrix` key of a 3x3 capacitance matrix with a coupler as the last port.

    Returns:
        loss: MSE loss to target ``C``
        C_Sigma: absolute value for :math:`C_\Sigma`
    """
    def _reciprocal_sum(*args):
        return 1 / sum((1 / e for e in args))

    C_sim = data['CMatrix']
    C_first_island = C_sim[0][0] + _reciprocal_sum(C_sim[0][2], C_sim[2][2])
    C_second_island = C_sim[1][1] + _reciprocal_sum(C_sim[1][2], C_sim[1][2])
    C_sigma = C_sim[0][1] + _reciprocal_sum(C_first_island, C_second_island)
    return ((C - C_sigma) * 1e15) ** 2, C_sigma


def loss_capacitance(data, C, i=0, j=0):
    r"""Loss function :math:`$\mathcal(L)` to get certain capacitance.

    Arguments:
        data: simulation results dictionary containing ``CMatrix``
        C: desired capacitance in farads
        i: first index for the desired capacitance from the ``CMatrix``
        j: second index for the desired capacitance from the ``CMatrix``

    Returns:
        loss: The computed loss
        x: Value with which the loss was computed

    Usage:

        .. code-block:: python

            from functools import partial
            loss = partial(lossC, C=50e-15, i=1, j=0)
            # Do the following inside the event loop
            results = load_results(dir_path)
            score = loss(results)
    """
    C_sim = data['CMatrix'][i][j]
    return ((C - C_sim) * 1e15) ** 2, C_sim  # Optimiser might get confused by small numbers

Select your loss function to use

In [8]:
load_results = load_results_capacitance

# loss = partial(loss_capacitance, C=100e-15, i=0, j=1)
loss = partial(loss_C_Sigma_two_islands, C=65e-15)

# load_results = load_results_Q
# loss = loss_Q

## Optimisation settings

Specify parameter search spaces, optimiser (such as ``HyperOptSearch``), possible scheduler, maximum iterations,
and parallel workers (only with Elmer).

In [9]:
search_config = {
    'params': {
        'island1_extent_[0]': tune.uniform(200, 700),
        'island1_extent_[1]': tune.uniform(50, 500),

        'island2_extent_[0]': tune.uniform(200, 700),
        'island2_extent_[1]': tune.uniform(50, 300),

        'coupler_extent[0]': tune.uniform(40, 500),
        'coupler_extent[1]': tune.uniform(20, 40),
        'coupler_offset': tune.uniform(5, 50),

        'ground_gap_[0]': tune.uniform(550, 1000),
        'ground_gap_[1]': tune.uniform(400, 1000),

        'squid_offset': tune.uniform(-20, 0),
        'island1_r': tune.uniform(0, 50),
        'island2_r': tune.uniform(0, 50),
    },
}

def constraints(trial):
    """ Return Sequence of (soft) constraints for the optimiser. Positive values are punished and <= 0 are ok.
    See optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.BoTorchSampler.html for details.
    """
    c0 = trial.params['params/ground_gap'] + trial.params['params/r_outer'] - 150
    c1 = trial.params['params/r_inner'] - trial.params['params/r_outer']  # negative is ok
    return [c0, c1]  # tries to optimise equal to 0

# Avoid more pickling of KLayout classes and don't show these as search params
create_and_export = partial(
    create_and_export_sim,
    sim_parameters=sim_parameters,
    export_parameters=export_parameters,
    sim_class=(sim_class.__module__, sim_class.__name__),  # we avoid pickling the class by loading this way
)

# Import chosen optimiser and scheduler. NB: Selecting a new optimiser often requires you to install a separate package
# See https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html

# pylint: disable=wrong-import-position,ungrouped-imports
from ray.tune.search.optuna import OptunaSearch
import optuna
import joblib  # shold come with Optuna


tune_config = tune.TuneConfig(
    metric='score',
    mode='min',
    search_alg=(optuna_search := OptunaSearch(
        # If you want soft constraints
        sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),
        # sampler=optuna.integration.BoTorchSampler(),
        metric=['score'],  #, 'outer_island_width'],
        mode=['min']  #, 'min'],
    )),
    ## Examples of other possible optimisers (search algorihtms)
    # search_alg=BayesOptSearch(),
    # search_alg=HEBOSearch(),
    # search_alg=DragonflySearch(
    #     metric='score',
    #     mode='min',
    #     optimizer='bandit',
    #     domain='Euclidean'
    # ),
    max_concurrent_trials=n_workers,  # Has to be 1 for Ansys, may be more for Elmer
    num_samples=100,  # max iterations, can be -1 for infinite
    time_budget_s=0,  # Time after which optimisation is stopped. May be useful along with ``num_samples=-1``.
)

C:\Users\NikoSavola\AppData\Local\Temp\ipykernel_15772\3628051455.py:51: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),


In [10]:
def event_loop(config):

    dir_path = create_and_export(**config, i=ray.air.session.get_trial_id())
    use_ansys = 'ansys_tool' in create_and_export.keywords['export_parameters']

    # Combine parameters with the following format to single array param:
    #   param_[0] + param_[1] -> param = [param_[0], param_[1]]
    arr_match = r'_\[(\d)\]'
    config['params'] |= {
        (arr_key := re.sub(arr_match, '', key)): config['params'].get(arr_key, []) + [config['params'][key]]
        for key in config['params']
        if re.match(arr_match, key)
    }

    subprocess.run(
        ['simulation.bat'] if use_ansys else [sh_interpreter, '-c', './simulation.sh'],
        cwd=dir_path,
        shell=True,
        check=True,
        startupinfo=STARTUPINFO
    )

    results = load_results(dir_path)
    score, x = loss(results)
    return {'score': score, 'value': x}
    # For multi-objective, just return the different loss functions and specify in `metric`, e.g.
    # from math import prod
    # return {'score': score, 'value': x, 'island1_size': math.prod(config['params']['island1_extent'])}

## Run optimisation

Start a Tune run and print the results.

In [11]:
tuner = tune.Tuner(
    tune.with_resources(event_loop, {"cpu": n_processes}),  # maximum resources given to worker
    param_space=search_config,
    tune_config=tune_config,
    run_config=ray.air.RunConfig(
        local_dir=os.environ['KQC_TMP_PATH'] + "/ray_results",  # needed for tensorboard
        checkpoint_config=ray.air.CheckpointConfig(
            checkpoint_frequency=1
        ),
        log_to_file=True
    )
)

# For resuming a run
# tuner = tune.Tuner.restore(os.environ['KQC_TMP_PATH'] + "/ray_results" + "/some_folder", restart_errored=False)

results = tuner.fit()

# This is specific only for the Optuna optimiser, we also save the 'study' object
study_file = Path(results._experiment_analysis.runner_data()['_local_checkpoint_dir']) / "study.pkl"
if study_file.exists():
    print(f'Loading study from {study_file}')
    study = joblib.load(study_file)
else:
    print(f'Saving study to {study_file}')
    study = optuna_search._ot_study
    joblib.dump(study, study_file)

2022-11-29 10:30:48,747	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-29 10:30:50,138	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\ray\tune\search\optuna\optuna_search.py:682: FutureWarning: UniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.UniformDistribution(
[I 2022-11-29 10:30:50,144] A new study created in memory with name: optuna


Trial name,status,loc,params/coupler_ex...,params/coupler_ex...,params/coupler_of...,params/ground_gap...,params/ground_gap...,params/island1_ex...,params/island1_ex...,params/island1_r,params/island2_ex...,params/island2_ex...,params/island2_r,params/squid_offset,iter,total time (s),score,value
event_loop_21539aa7,TERMINATED,127.0.0.1:38136,362.05,27.7159,15.5868,973.961,441.591,681.788,423.208,0.42293,514.838,102.249,24.498,-4.61943,1,24.8626,2.37865,6.65423e-14
event_loop_22b783f1,TERMINATED,127.0.0.1:5488,465.326,38.3152,38.159,566.491,703.455,630.985,445.328,9.92951,572.705,213.399,3.62348,-8.38392,1,22.744,1.52924,6.62366e-14
event_loop_22b991e1,TERMINATED,127.0.0.1:1424,92.3451,21.1561,28.873,905.427,446.146,398.38,291.633,35.9106,426.923,68.7228,34.7804,-1.04736,1,23.7801,0.065216,6.47446e-14
event_loop_22bccf4a,TERMINATED,127.0.0.1:30288,479.531,30.7543,33.9212,552.108,467.219,346.91,91.4031,11.0482,453.453,131.78,16.251,-10.4888,1,23.7771,1.12514,6.60607e-14
event_loop_22c0a613,TERMINATED,127.0.0.1:12740,288.839,22.9455,19.1171,966.146,800.793,684.792,363.806,0.00478136,212.74,221.388,37.9161,-5.22573,1,23.7731,0.669629,6.58183e-14
event_loop_318f32af,TERMINATED,127.0.0.1:35908,442.769,20.1371,6.83246,974.214,814.018,661.414,402.099,26.4769,377.091,60.8694,15.425,-2.57216,1,25.7501,3.41413,6.68477e-14
event_loop_332984b6,TERMINATED,127.0.0.1:5488,65.8398,24.3925,34.85,651.8,497.293,381.666,66.3513,48.8419,561.822,208.017,26.6252,-4.35451,1,26.9659,0.208001,6.45439e-14
event_loop_333284fb,TERMINATED,127.0.0.1:30288,207.344,27.5178,21.2576,731.661,602.377,492.772,405.752,46.9152,256.949,192.749,14.0724,-13.2143,1,27.0158,0.070077,6.52647e-14
event_loop_33389f90,TERMINATED,127.0.0.1:1424,161.011,20.0694,30.881,580.21,774.275,506.177,93.9779,16.2054,285.699,182.745,41.6493,-9.21901,1,26.9782,0.0043803,6.50662e-14
event_loop_3342ffc0,TERMINATED,127.0.0.1:16688,178.115,31.0078,25.8241,854.177,787.779,585.714,454.118,27.1252,603.953,73.1505,39.6789,-1.62724,1,24.104,0.00898951,6.50948e-14


C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=700.0, low=200.0) is deprecated and internally converted to FloatDistribution(high=700.0, log=False, low=200.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=500.0, low=50.0) is deprecated and internally converted to FloatDistribution(high=500.0, log=False, low=50.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=300.0, low=50.0) is deprecated and internally converted to FloatDistribution(high=300.0, log=False, low=50.0, step=None). See https://github.com/optuna/optuna/issues/294

(event_loop pid=38136) Submitting the main script of simulation 1/1
(event_loop pid=38136) --------------------------------------------
(event_loop pid=38136) Simulation 1/1 Gmsh


(event_loop pid=5488) Ground location of port 1 is not determined.
(event_loop pid=5488) Ground location of port 2 is not determined.
(event_loop pid=30288) Ground location of port 1 is not determined.
(event_loop pid=30288) Ground location of port 2 is not determined.
(event_loop pid=1424) Ground location of port 1 is not determined.
(event_loop pid=1424) Ground location of port 2 is not determined.
(event_loop pid=12740) Ground location of port 1 is not determined.
(event_loop pid=12740) Ground location of port 2 is not determined.


...

Result for event_loop_5a4566ce:
  date: 2022-11-29_10-40-00
  done: false
  experiment_id: 8369bc422c2546b0bfa1fa2ae6a3f957
  hostname: 8CG2026VMQ
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 32904
  score: 0.0015960273736840929
  time_since_restore: 24.908171892166138
  time_this_iter_s: 24.908171892166138
  time_total_s: 24.908171892166138
  timestamp: 1669711200
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5a4566ce
  value: 6.496004968869102e-14
  warmup_time: 0.003999471664428711
  
Result for event_loop_58141e38:
  date: 2022-11-29_10-40-00
  done: false
  experiment_id: 099bbb56100a4853ab25dcecee0df43e
  hostname: 8CG2026VMQ
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 34212
  score: 0.01866387143269772
  time_since_restore: 25.561833143234253
  time_this_iter_s: 25.561833143234253
  time_total_s: 25.561833143234253
  timestamp: 1669711200
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 58141e38
  value: 6.51366157803

2022-11-29 10:40:01,520	WARNING optuna_search.py:549 -- 'params/ground_gap'


Result for event_loop_5abb740e:
  date: 2022-11-29_10-40-01
  done: false
  experiment_id: 069fb179c74d4aca8e68ee141be75dda
  hostname: 8CG2026VMQ
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 20584
  score: 0.04167876717639455
  time_since_restore: 20.744935274124146
  time_this_iter_s: 20.744935274124146
  time_total_s: 20.744935274124146
  timestamp: 1669711201
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5abb740e
  value: 6.479584621684526e-14
  warmup_time: 0.03694653511047363
  
Result for event_loop_5abb740e:
  date: 2022-11-29_10-40-01
  done: true
  experiment_id: 069fb179c74d4aca8e68ee141be75dda
  experiment_tag: 100_coupler_extent_0=138.7183,coupler_extent_1=27.7920,coupler_offset=35.4457,ground_gap__0=590.7293,ground_gap__1=810.0079,island1_extent__0=301.2301,island1_extent__1=212.1680,island1_r=45.8115,island2_extent__0=498.6821,island2_extent__1=201.2392,island2_r=22.6392,squid_offset=-11.8371
  hostname: 8CG2026VMQ
  iterations_since_resto

2022-11-29 10:40:01,694	INFO tune.py:758 -- Total run time: 551.56 seconds (551.34 seconds for the tuning loop).


Saving study to D:\LocalSimulations\ray_results\event_loop_2022-11-29_10-30-46\study.pkl


This is specific only for the Optuna optimiser

In [12]:
# Default 'importance' metric is fANOVA, see optuna.importance.FanovaImportanceEvaluator
fig = optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0])
fig.show()

fig = optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[0])
fig.show()

try:
    fig = optuna.visualization.plot_pareto_front(study)  # or optuna.visualization.matplotlib.plot_pareto_front(study)
    fig.show()
except ValueError as e:
    print('Not multi-objective.', e)

C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:69: UserWarning: `target` is specified, but `target_name` is the default value, 'Objective Value'.
  warnings.warn(


C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



Not multi-objective. `plot_pareto_front` function only supports 2 or 3 objective studies when using `targets` is `None`. Please use `targets` if your objective studies have more than 3 objectives.


In [13]:
df = results.get_dataframe()
best = results.get_best_result(metric="score", mode="min")

display(df.filter(regex='(score|value|time_this|config)'))
print(f"{best.metrics['value']=}", best.config)

,score,value,time_this_iter_s,config/params/coupler_extent[0],config/params/coupler_extent[1],config/params/coupler_offset,config/params/ground_gap_[0],config/params/ground_gap_[1],config/params/island1_extent_[0],config/params/island1_extent_[1],config/params/island1_r,config/params/island2_extent_[0],config/params/island2_extent_[1],config/params/island2_r,config/params/squid_offset
0,2.378650,6.654229e-14,24.862607,362.050000,27.715947,15.586779,973.960821,441.590983,681.788488,423.208469,0.422930,514.837858,102.249439,24.498041,-4.619429
1,1.529243,6.623663e-14,22.743998,465.326324,38.315150,38.158982,566.490504,703.455020,630.985211,445.327583,9.929512,572.704804,213.399178,3.623480,-8.383924
2,0.065216,6.474463e-14,23.780132,92.345077,21.156135,28.872983,905.427031,446.145626,398.380231,291.633034,35.910578,426.923309,68.722849,34.780419,-1.047359
3,1.125141,6.606073e-14,23.777133,479.530626,30.754263,33.921167,552.107750,467.219007,346.910241,91.403128,11.048207,453.452599,131.779683,16.250969,-10.488841
4,0.669629,6.581831e-14,23.773132,288.839116,22.945484,19.117097,966.146159,800.793105,684.792297,363.806113,0.004781,212.739615,221.388228,37.916134,-5.225726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.003597,6.505997e-14,23.587928,170.052123,23.593087,26.496228,633.439507,937.367093,566.086146,173.886206,15.237389,267.913197,169.227401,44.095621,-10.644932
96,0.012709,6.511273e-14,24.907390,199.498914,28.390554,18.668650,680.662886,658.349400,370.092697,340.864310,50.000000,235.487498,205.897452,18.384754,-15.549671
97,0.018664,6.513662e-14,25.561833,196.043196,28.278428,18.816662,681.913146,646.746466,370.091675,339.749781,49.873739,229.916508,206.182219,18.334185,-15.571778
98,0.001596,6.496005e-14,24.908172,198.055304,28.359856,18.700302,681.883819,655.018246,371.616026,342.484092,49.775752,234.953861,206.032469,18.396706,-15.536233


best.metrics['value']=6.499782486793626e-14 {'params': {'island1_extent_[0]': 312.07040396112023, 'island1_extent_[1]': 161.96389029443867, 'island2_extent_[0]': 482.68741391633813, 'island2_extent_[1]': 207.6941801947922, 'coupler_extent[0]': 182.43505144550315, 'coupler_extent[1]': 25.52931941982596, 'coupler_offset': 31.474025149199303, 'ground_gap_[0]': 660.1880061411956, 'ground_gap_[1]': 807.937349652368, 'squid_offset': -11.802296253011836, 'island1_r': 33.46565195311718, 'island2_r': 32.447869961890596}}
